### 🐁 AmadeusGPT Quickstart

---


#### Notebook Overview:
1. Bootstrap the Jupyter interpreter 🛠️
2. Load secrets & config 🔑
3. Project initialization 📁
4. Run a demo query 🎬
5. Appendix 📚

#### 1 · Bootstrap the Jupyter interpreter 🛠️
-  Run the below cell *once* per fresh clone or when dependencies change
-  ⚠️ Select a 3.10 Python environment before you run the below code (if you already have the AmadeusGPT Jupyter kernel select that and skip to [Jump to “Load secrets & config”](#load-secrets-config)
)
<a id="bootstrap-jupyter-interpreter"></a>

In [ ]:
%pip install --quiet --upgrade pip
%pip install --quiet -e .[dev]            # editable install → reads pyproject.toml
%pip install --quiet ipykernel       # needed for kernel registration
%matplotlib inline

Register the interpreter as a named Jupyter kernel - unless you already have access to the AmadeusGPT (venv)

In [ ]:
!python -m ipykernel install --user --name amadeus-gpt-env --display-name "AmadeusGPT (venv)"

print("✅  Bootstrapping done – now **Restart** the kernel (Kernel ▶ Restart) and select “AmadeusGPT (venv)” in the top-right kernel picker.")

<a id="load-secrets-config"></a>
#### 2 · Load secrets & config 🔑


In [2]:
from dotenv import load_dotenv, find_dotenv

env_path = find_dotenv()
load_dotenv(env_path, override=True)
print("🔐  Environment loaded from:", env_path or "❌  .env file not found. Create one and add your OPENAI_API_KEY='api_key' to complete this step")


🔐  Environment loaded from: c:\Users\hugos\Desktop\AmadeusGPT\.env


#### 3 · Project initialisation 📁
You should re-run this each time you (re-)open this program.

In [1]:
import os
from amadeusgpt import AMADEUS, create_project
from amadeusgpt.utils import parse_result

# ── resolve repo root dynamically ──
ROOT = os.getcwd()

DATA_DIR   = os.path.join(ROOT, "data")
RESULT_DIR = os.path.join(ROOT, "results")

project_cfg = create_project(
    data_folder   = DATA_DIR,
    result_folder = RESULT_DIR,
    **{
        "data_info.video_suffix": ".avi",
        "keypoint_info.use_3d"  : True,
    }
)

amadeus = AMADEUS(project_cfg)
print("✅  Project ready:", project_cfg)


Project created at c:\Users\hugos\Desktop\AmadeusGPT\results. Results will be saved to c:\Users\hugos\Desktop\AmadeusGPT\results
The project will load video files (*.avi) and optionally keypoint files from c:\Users\hugos\Desktop\AmadeusGPT\data
A copy of the project config file is saved at c:\Users\hugos\Desktop\AmadeusGPT\results\config.yaml
{'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data',
               'result_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\results',
               'video_suffix': '.avi'},
 'keypoint_info': {'include_confidence': False, 'use_3d': True},
 'llm_info': {'gpt_model': 'gpt-4o',
              'keep_last_n_messages': 2,
              'max_tokens': 4096,
              'temperature': 0.0},
 'object_info': {'load_objects_from_disk': False, 'use_grid_objects': False},
 'video_info': {'scene_frame_number': 1}}
✅  Project ready: {'data_info': {'data_folder': 'c:\\Users\\hugos\\Desktop\\AmadeusGPT\\data', 'result_folder': 'c:\\Users\\

#### 4 · Run a demo query 🎬


### Appendix 📚

---

In [5]:
from pathlib import Path
import json, traceback

queries_path = Path("queries.json")
with queries_path.open(encoding="utf‑8") as f:
    queries = json.load(f)

TEMPLATE = (
    "Follow the definition below *verbatim*.  "
    "Identify every **continuous bout** that satisfies the rules then return ONLY the list of (start_frame, end_frame) tuples "
    "using 0‑based indexing. Do not add explanations or any other text.\n\n"
)

for key, definition in queries.items():
    try:
        prompt = (
            TEMPLATE
            + definition
            + f"\n\nAfter processing, save the tuple list to 'results/{key}.txt'."
        )

        print(f"🔍  Querying '{key}'…")
        msg = amadeus.step(prompt)
        parse_result(amadeus, msg)

    except Exception as e:
        print(f"❌  Error processing '{key}': {e}")
        traceback.print_exc()
        continue


# if INCLUDE_GRAPH:
#     try:
#         print(f"🔍  Generating summary graph: {GRAPH_QUERY}")
#         msg_graph = amadeus.step(GRAPH_QUERY)
#         parse_result(amadeus, msg_graph)
#     except Exception as e:
#         print(f"❌  Error processing graph: {e}")
#         traceback.print_exc()


🔍  Querying 'locomotion'…


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


doing active forgetting


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


current total cost 0.2843 $
current input tokens 4688
current accumulated tokens 41734


...

current total cost 0.4157 $
current input tokens 3914
current accumulated tokens 61290


```python
def identify_head_rotation_bouts(identifier):
    """
    Identify continuous bouts of head rotation based on the given criteria.
    
    Parameters
    ----------
    identifier: Identifier
        Contains information about the video, keypoint, and config.
    
    Returns
    -------
    List[Tuple[int, int]]
        A list of tuples representing the start and end frames of each head rotation bout.
    """
    import numpy as np

    def signed_angle(v1, v2):
        """Calculate the signed angle between two vectors."""
        angle = np.arctan2(v2[1], v2[0]) - np.arctan2(v1[1], v1[0])
        return np.degrees(angle)

    # Create an analysis instance
    analysis = create_analysis(identifier)
    
    # Get keypoints and keypoint names
    keypoints = analysis.get_keypoints()  # shape: (n_frames, n_individuals, n_kpts, n_dims)
    keypoint_names = analysis.get_keypoint_names()
    
    # Get indices for required keypoints
    left_ear_idx = keypoint_names.index('Left ear')
    right_ear_idx = keypoint_names.index('Right ear')
    snout_idx = keypoint_names.index('Snout')
    tail_base_idx = keypoint_names.index('Tail base')
    
    # Extract relevant keypoints for the single animal
    left_ear = keypoints[:, 0, left_ear_idx, :2]  # shape: (n_frames, 2)
    right_ear = keypoints[:, 0, right_ear_idx, :2]  # shape: (n_frames, 2)
    snout = keypoints[:, 0, snout_idx, :2]  # shape: (n_frames, 2)
    tail_base = keypoints[:, 0, tail_base_idx, :2]  # shape: (n_frames, 2)
    
    # Calculate head vector and body orientation vector
    head_vector = right_ear - left_ear  # shape: (n_frames, 2)
    body_orientation = snout - tail_base  # shape: (n_frames, 2)
    
    # Calculate angles
    angles = np.array([signed_angle(head_vector[i], body_orientation[i]) for i in range(len(head_vector))])  # shape: (n_frames,)
    
    # Calculate angle differences over 6 frames
    angle_diffs = np.abs(np.diff(angles, n=6, prepend=angles[0]))  # shape: (n_frames,)
    body_orientation_diffs = np.abs(np.diff([signed_angle(body_orientation[i], body_orientation[i+1]) for i in range(len(body_orientation)-1)], prepend=0))  # shape: (n_frames-1,)
    
    # Initialize variables
    in_bout = False
    bouts = []
    n = len(angles)
    
    # Loop through frames
    for i in range(n):
        if i < len(body_orientation_diffs) and angle_diffs[i] >= 40 and body_orientation_diffs[i] < 10:
            if not in_bout:
                in_bout = True
                start = i
        elif in_bout and (i >= len(body_orientation_diffs) or angle_diffs[i] < 40 or body_orientation_diffs[i] >= 10):
            if i - start >= 1:
                bouts.append((start, i - 1))
            in_bout = False
    
    # Check if a bout was ongoing at the end
    if in_bout and (n - start) >= 1:
        bouts.append((start, n - 1))
    
    # Merge bouts separated by < 4 frames
    merged_bouts = []
    if bouts:
        start, end = bouts[0]
        for i in range(1, len(bouts)):
            next_start, next_end = bouts[i]
            if next_start - end < 4:
                end = next_end
            else:
                merged_bouts.append((start, end))
                start, end = next_start, next_end
        merged_bouts.append((start, end))
    
    # Save results to file
    with open('results/head rotations.txt', 'w') as f:
        for bout in merged_bouts:
            f.write(f"{bout[0]},{bout[1]}\n")
    
    return merged_bouts
```

error occurs in code execution
Traceback (most recent call last):
  File "c:\Users\hugos\Desktop\AmadeusGPT\.venv\lib\site-packages\amadeusgpt\programs\sandbox.py", line 350, in code_execution
    exec(f"result = {call_str}", namespace)
  File "<string>", line 1, in <module>
  File "<string>", line 60, in identify_head_rotation_bouts
IndexError: index 10794 is out of bounds for axis 0 with size 10794

the code that gave errors was 
def identify_head_rotation_bouts(identifier):
    """
    Identify continuous bouts of head rotation based on the given criteria.
    
    Parameters
    ----------
    identifier: Identifier
        Contains information about the video, keypoint, and config.
    
    Returns
    -------
    List[Tuple[int, int]]
        A list of tuples representing the start and end frames of each head rotation bout.
    """
    import numpy as np

    def signed_angle(v1, v2):
        """Calculate the signed angle between two vectors."""
        angle = np.arctan2(v2[1], 

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


current total cost 0.4375 $
current input tokens 1165
current accumulated tokens 63524
```python
def identify_head_rotation_bouts(identifier):
    """
    Identify continuous bouts of head rotation based on the given criteria.
    
    Parameters
    ----------
    identifier: Identifier
        Contains information about the video, keypoint, and config.
    
    Returns
    -------
    List[Tuple[int, int]]
        A list of tuples representing the start and end frames of each head rotation bout.
    """
    import numpy as np

    def signed_angle(v1, v2):
        """Calculate the signed angle between two vectors."""
        angle = np.arctan2(v2[1], v2[0]) - np.arctan2(v1[1], v1[0])
        return np.degrees(angle)

    # Create an analysis instance
    analysis = create_analysis(identifier)
    
    # Get keypoints and keypoint names
    keypoints = analysis.get_keypoints()  # shape: (n_frames, n_individuals, n_kpts, n_dims)
    keypoint_names = analysis.get_keypoint_names()
    
 


#### 1. Updating code  
- After you’ve made changes to your package source, simply re-run the dependency‐install cell. [Re-install dependencies here.](#bootstrap-jupyter-interpreter)
#### 2. Troubleshooting

* **Kernel name not visible?**

  * Reload VS Code/Notebook window (e.g. ⌘⇧P → “Developer: Reload Window”).
  * Make sure you're selecting Jupyter Kernels not Python Environments to find `AmadeusGPT (venv)`
  * Ensure the workspace interpreter is set to your `.venv` (click the Python version in the status bar).

* **Version or Downloading Issues?**
  
  * Versioning errors occur if your Kernel is not Python version 3.10 (required for AmadeusGPT).
    *  Ensure that you have this version of Python installed and have selected it as the version for your environment
    *  You can check this in top right corner. If you see it's Python 3.10... that's not the issue.

* **“Package X not found” after reinstalling**

  * Confirm you’ve restarted the kernel (Kernel ▶ Restart).
  * Run `pip list` in a cell to verify that your package is installed in the active environment.

* **Still stuck?**

  * In a terminal, run:

    ```bash
    jupyter kernelspec list
    ```

    to verify that `amadeus-gpt-env` appears under your user kernels.
  * Stop & restart the Jupyter server
    (in VS Code: ⌘⇧P → “Jupyter: Stop Jupyter Server”, then run any cell again).